<a href="https://colab.research.google.com/github/Daryldactyl/First_ANN/blob/main/Bank_ANN(binary_class).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artificial Neural Network

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
#What version tensorflow are we using?
tf.__version__

'2.15.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [7]:
df = pd.read_csv("https://raw.githubusercontent.com/Daryldactyl/First_ANN/Files/Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [9]:
#Checking for null values in the data
df.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [16]:
#Checking for imbalance
df['Exited'].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [19]:
print(f'Model gives a {(7963/10000)*100}% accuracy if it guesses all 0')

Model gives a 79.63% accuracy if it guesses all 0


In [36]:
from sklearn.utils import resample
majority_class = df[df['Exited'] == 0]
minority_class = df[df['Exited'] == 1]
minority_upsample = resample(minority_class, replace=True, n_samples=majority_class.shape[0], random_state=0)
df = pd.concat([majority_class, minority_upsample])
df['Exited'].value_counts()

0    7963
1    7963
Name: Exited, dtype: int64

In [37]:
x = df.iloc[:, 3:-1].values
y = df.iloc[:, -1].values

### Encoding categorical data

Label Encoding the "Gender" column

In [38]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:, 2] = le.fit_transform(x[:, 2])

One Hot Encoding the "Geography" column

In [39]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
x = np.array(ct.fit_transform(x))

### Splitting the dataset into the Training set and Test set

In [40]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, random_state=0)

### Feature Scaling

In [41]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [42]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [43]:
#This was made for experimentation with the first ANN, Can use a param grid to find ideal number of layers/neurons then ideal activation function
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the second hidden layer

In [44]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer

In [45]:
#Since the output is binary you only need 1 node that either reads 'on' or 'off' which is either 0 or 1
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) #sigmoid gives values between 0 and 1

## Part 3 - Training the ANN

### Compiling the ANN

In [46]:
ann.compile(optimizer = 'adam',
            loss = 'binary_crossentropy',
            metrics = ['accuracy']
)

### Training the ANN on the Training set

In [47]:
ann.fit(x_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
399/399 [==============================] - 2s 3ms/step - loss: 0.6790 - accuracy: 0.5450
Epoch 2/100
399/399 [==============================] - 2s 5ms/step - loss: 0.6071 - accuracy: 0.6774
Epoch 3/100
399/399 [==============================] - 2s 6ms/step - loss: 0.5816 - accuracy: 0.7028
Epoch 4/100
399/399 [==============================] - 2s 5ms/step - loss: 0.5711 - accuracy: 0.7100
Epoch 5/100
399/399 [==============================] - 2s 4ms/step - loss: 0.5632 - accuracy: 0.7156
Epoch 6/100
399/399 [==============================] - 2s 4ms/step - loss: 0.5573 - accuracy: 0.7152
Epoch 7/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5519 - accuracy: 0.7194
Epoch 8/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5462 - accuracy: 0.7224
Epoch 9/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5393 - accuracy: 0.7296
Epoch 10/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5309 - accura

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer?

**Solution**

In [48]:
print(ann.predict(sc.transform([[1.0, 0.0, 0.0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 [==============================] - 0s 59ms/step
[[False]]


Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

In [49]:
y_pred = ann.predict(x_test)
#This will give percentages so we need to set up for binary:
y_pred = (y_pred > 0.5)

100/100 [==============================] - 0s 1ms/step


### Making the Confusion Matrix

In [50]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1292  286]
 [ 423 1185]]


0.7774639045825487

In [51]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.82      0.78      1578
           1       0.81      0.74      0.77      1608

    accuracy                           0.78      3186
   macro avg       0.78      0.78      0.78      3186
weighted avg       0.78      0.78      0.78      3186

